### Ingestion del archivo "language_role.json"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col
from pyspark.sql.types import StructType,StructField,IntegerType,StringType


In [0]:
languages_roles_schema = StructType([
    StructField("roleId", IntegerType(), True),
    StructField("languageRole", StringType(), True)
    ])


In [0]:
languages_roles_df = spark.read.schema(languages_roles_schema).option("multiLine", True).json(f"{bronze_folder_path}/{v_file_date}/language_role.json")#Activo con el option la lectura multilinea del json

In [0]:
languages_roles_df.printSchema()

In [0]:
display(languages_roles_df)

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "roleId" renombrar a "role_id"
2. "languageRole" renombrar a "language_role"
3. Agregar las columnas "ingestion_date" y "enviroment"

In [0]:
languages_roles_final_df = add_ingestion_date(languages_roles_df).withColumnsRenamed({"roleId":"role_id","languageRole":"language_role"}).withColumns({"enviroment":lit(v_enviroment),"file_date":lit(v_file_date)})

In [0]:
display(languages_roles_final_df)

#### Paso 4 - Escribir la salida en formato "Parquet"

In [0]:
languages_roles_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.languages_roles")

In [0]:
%fs
ls /mnt/historialpeliculas/silver/languages_roles/

In [0]:
%sql
SELECT *
FROM MOVIE_SILVER.LANGUAGES_ROLES;

In [0]:
dbutils.notebook.exit("Success")